In [6]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#Plot setup

fig = plt.figure()
ax_rgb = fig.add_subplot(111, projection='3d')
ax_rgb.set_box_aspect([1, 1, 1])
t = np.linspace(0.0, 1.0, 2)

#Achsen gleich skalieren
def set_axes_equal(ax):
    '''Gleiche Skalierung für alle 3 Achsen'''
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    y_range = abs(y_limits[1] - y_limits[0])
    z_range = abs(z_limits[1] - z_limits[0])

    max_range = max([x_range, y_range, z_range]) / 2

    x_middle = np.mean(x_limits)
    y_middle = np.mean(y_limits)
    z_middle = np.mean(z_limits)

    ax.set_xlim3d([x_middle - max_range, x_middle + max_range])
    ax.set_ylim3d([y_middle - max_range, y_middle + max_range])
    ax.set_zlim3d([z_middle - max_range, z_middle + max_range])



#Ausrichtung wie in solidworks/3d kamerakonventionen
ax_rgb.view_init(elev=90, azim=-90)

for r in range(0,255,30):
    for g in range(0,255,30):
        for b in range(0,255,30):
            ax_rgb.scatter3D([r], [g], [b], marker='o', color=[r / 255, g / 255, b / 255])
def plotcube():
    # Ursprungswürfel-Koordinaten
    r = [100, 255]
    g = [0, 80]
    b = [0, 80]

    # Alle Eckpunkte generieren
    cube_points = np.array([[xi, yi, zi] for xi in r for yi in g for zi in b])

    # Linienpaare für die Würfelkanten
    edges = [
        (0,1), (0,2), (0,4),
        (3,1), (3,2), (3,7),
        (5,1), (5,4), (5,7),
        (6,2), (6,4), (6,7),
        (3,6)
    ]

    # Würfel plotten
    for i,j in edges:
        ax_rgb.plot(
            [cube_points[i,0], cube_points[j,0]],
            [cube_points[i,1], cube_points[j,1]],
            [cube_points[i,2], cube_points[j,2]],
            color="gray"
        )
plotcube()

# Axes labels and legend
ax_rgb.set_xlabel("X")
ax_rgb.set_ylabel("Y")
ax_rgb.set_zlabel("Z")
plt.show()


<IPython.core.display.Javascript object>

In [2]:
r = [100, 255]
g = [0, 80]
b = [0, 80]

# Alle Eckpunkte generieren
cube_points = np.array([[xi, yi, zi] for xi in r for yi in g for zi in b])
print(cube_points)

[[100   0   0]
 [100   0  80]
 [100  80   0]
 [100  80  80]
 [255   0   0]
 [255   0  80]
 [255  80   0]
 [255  80  80]]


In [9]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Line3DCollection

# Plot setup
fig = plt.figure()
ax_rgb = fig.add_subplot(111, projection='3d')
ax_rgb.set_xlim3d([0, 255])
ax_rgb.set_ylim3d([0, 255])
ax_rgb.set_zlim3d([0, 255])
ax_rgb.set_box_aspect([1, 1, 1])

# Punkte im RGB-Raum erzeugen (0–255 in 30er Schritten + 255 als Endpunkt)
h = 10
r = np.unique(np.r_[np.arange(0,256,h), 255])
g = np.unique(np.r_[np.arange(0,256,h), 255])
b = np.unique(np.r_[np.arange(0,256,h), 255])



R, G, B = np.meshgrid(r, g, b, indexing="ij")
points = np.stack([R.ravel(), G.ravel(), B.ravel()], axis=1)
colors = points / 255.0
mask = (points[:, 0] > 99) & (points[:, 1] < 81) & (points[:, 2] < 81)
"""points = points[mask]
colors = colors[mask]"""
# Alle Punkte auf einmal plotten
ax_rgb.scatter(points[:, 0], points[:, 1], points[:, 2], s=4, c=colors, marker='o')

# Box-Funktion
def add_box(ax, lo, hi, color='gray', lw=1.0):
    x0,y0,z0 = lo
    x1,y1,z1 = hi
    verts = np.array([
        [x0,y0,z0],[x1,y0,z0],
        [x0,y1,z0],[x1,y1,z0],
        [x0,y0,z1],[x1,y0,z1],
        [x0,y1,z1],[x1,y1,z1],
    ])
    edges = [(0,1),(0,2),(0,4),
             (3,1),(3,2),(3,7),
             (5,1),(5,4),(5,7),
             (6,2),(6,4),(6,7)]
    segs = [(verts[i], verts[j]) for i,j in edges]
    lc = Line3DCollection(segs, colors=[color], linewidths=lw)
    ax.add_collection3d(lc)

# Beispiel-Box einfügen
#add_box(ax, lo=[100,0,0], hi=[255,80,80], color='gray', lw=1.0)

# Achsen beschriften
ax_rgb.set_xlabel("R")
ax_rgb.set_ylabel("G")
ax_rgb.set_zlabel("B")

# Kameraperspektive ähnlich wie in SolidWorks
ax_rgb.view_init(elev=90, azim=-90)
plt.show()
print(points)

<IPython.core.display.Javascript object>

[[  0   0   0]
 [  0   0  10]
 [  0   0  20]
 ...
 [255 255 240]
 [255 255 250]
 [255 255 255]]


In [4]:
# GIF: erst alle Punkte, dann nur Maskenpunkte – jeweils mit Kameradrehung.
# Speichert *im aktuellen Notebook-Verzeichnis* als "Visualisierung RGB GIF.gif".

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

# --------- Daten ---------
h = 25  # Gitterabstand (geringe Rechenlast)
vals = np.unique(np.r_[np.arange(0, 256, h), 255])

R, G, B = np.meshgrid(vals, vals, vals, indexing="ij")
points = np.stack([R.ravel(), G.ravel(), B.ravel()], axis=1).astype(float)
colors = points / 255.0

# Maske (Beispiel)
mask = (points[:, 0] > 99) & (points[:, 1] < 81) & (points[:, 2] < 81)
pts_in, col_in = points[mask], colors[mask]
pts_out, col_out = points[~mask], colors[~mask]

# --------- Figure ---------
fig = plt.figure(figsize=(5, 5), dpi=400)
ax_rgb = fig.add_subplot(111, projection='3d')
ax_rgb.set_xlim3d(0, 255)
ax_rgb.set_ylim3d(0, 255)
ax_rgb.set_zlim3d(0, 255)
ax_rgb.set_box_aspect([1, 1, 1])
ax_rgb.set_axis_off()  # Achsen ausblenden

# Startansicht
elev0, azim0 = 30, -60
ax_rgb.view_init(elev=elev0, azim=azim0)

# Zwei Scatter: außerhalb (Phase A sichtbar), innerhalb (beide Phasen sichtbar)
scat_out = ax_rgb.scatter(pts_out[:, 0], pts_out[:, 1], pts_out[:, 2],
                          s=12, c=col_out, marker='o', depthshade=False)
scat_in = ax_rgb.scatter(pts_in[:, 0], pts_in[:, 1], pts_in[:, 2],
                         s=18, c=col_in, marker='o', depthshade=False)

# --------- Animation ---------
frames_A = 60  # Rundum-Kameradrehung über alle Punkte
frames_B = 60  # gleiche Kameradrehung, aber nur Maskenpunkte sichtbar
total_frames = frames_A + frames_B

def update(i):
    if i < frames_A:
        # Phase A: Kamera dreht, beide Punktwolken sichtbar
        scat_out.set_visible(True)
        az = azim0 + 360.0 * (i / frames_A)
        ax_rgb.view_init(elev=elev0, azim=az)
    else:
        # Phase B: gleiche Kameradrehung, aber NUR Maskenpunkte sichtbar
        scat_out.set_visible(False)
        j = i - frames_A
        az = azim0 + 360.0 * (j / frames_B)
        ax_rgb.view_init(elev=elev0, azim=az)
    return scat_out, scat_in

anim = FuncAnimation(fig, update, frames=total_frames, interval=50, blit=False)

# --------- Speichern ---------
output_path = "Visualisierung RGB GIF.gif"  # relativ -> Notebook-Verzeichnis
anim.save(output_path, writer=PillowWriter(fps=20))
print(f"GIF gespeichert: {output_path}")


<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [61]:
# GIF: erst langsame Kameradrehung über ALLE Punkte,
# dann weicher Übergang (von außen nach innen) zu NUR Maskenpunkten.
# Speichert im aktuellen Notebook-Verzeichnis als "Visualisierung RGB GIF.gif".

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

# ---------- Daten ----------
h = 255/10  # Gitterabstand (geringe Rechenlast)
vals = np.unique(np.r_[np.arange(0, 256, h), 255])

R, G, B = np.meshgrid(vals, vals, vals, indexing="ij")
points = np.stack([R.ravel(), G.ravel(), B.ravel()], axis=1).astype(float)
colors = points / 255.0

# Maske: Quader x ∈ [100,255], y ∈ [0,80], z ∈ [0,80]
x0, x1 = 100, 255
y0, y1 = 0, 80
z0, z1 = 0, 80
mask = (points[:,0] >= x0) & (points[:,0] <= x1) & \
       (points[:,1] >= y0) & (points[:,1] <= y1) & \
       (points[:,2] >= z0) & (points[:,2] <= z1)

pts_in, col_in = points[mask], colors[mask]
pts_out, col_out = points[~mask], colors[~mask]

# ---------- Entfernung zur Box (für "außen nach innen" Übergang) ----------
# euklidische Distanz eines Punkts zum Quader
def dist_to_box(p):
    dx = 0 if x0 <= p[0] <= x1 else (x0 - p[0] if p[0] < x0 else p[0] - x1)
    dy = 0 if y0 <= p[1] <= y1 else (y0 - p[1] if p[1] < y0 else p[1] - y1)
    dz = 0 if z0 <= p[2] <= z1 else (z0 - p[2] if p[2] < z0 else p[2] - z1)
    return (dx*dx + dy*dy + dz*dz) ** 0.5

if len(pts_out) > 0:
    dists = np.array([dist_to_box(p) for p in pts_out])
else:
    dists = np.array([])

# teile Außenpunkte in Schichten (außen -> innen)
num_layers = 12  # mehr Schichten = sanfterer Übergang
if len(dists) > 0:
    # Grenzen so wählen, dass die "äußersten" zuerst verschwinden
    edges = np.quantile(dists, np.linspace(0, 1, num_layers + 1))
    # Layer-Indices: höchster Abstand = Layer 0 (als erstes ausblenden)
    layer_ids = np.clip(num_layers - 1 - np.digitize(dists, edges[1:-1], right=True), 0, num_layers-1)
else:
    layer_ids = np.array([])

# ---------- Figure ----------
fig = plt.figure(figsize=(5, 5), dpi=1000)
ax_rgb = fig.add_subplot(111, projection='3d')
ax_rgb.set_xlim3d(0, 255)
ax_rgb.set_ylim3d(0, 255)
ax_rgb.set_zlim3d(0, 255)
ax_rgb.set_box_aspect([1, 1, 1])
ax_rgb.set_axis_off()  # Achsen ausblenden

# Startansicht (leicht schräg) – wird in Phase A langsam rotiert
elev0, azim0 = 30, -60
ax_rgb.view_init(elev=elev0, azim=azim0)

# Streuungen erstellen
scat_in = ax_rgb.scatter(pts_in[:, 0], pts_in[:, 1], pts_in[:, 2],
                         s=18, c=col_in, marker='o', depthshade=False)

# Außenpunkte in Layern (jede Schicht eigener Artist, damit wir sie stufenweise ausblenden können)
scat_layers = []
for lid in range(num_layers):
    idx = np.where(layer_ids == lid)[0]
    if idx.size > 0:
        P = pts_out[idx]
        C = col_out[idx]
        s = ax_rgb.scatter(P[:, 0], P[:, 1], P[:, 2], s=12, c=C, marker='o', depthshade=False)
        scat_layers.append(s)

# ---------- Animation ----------
# Phase A: langsame 360°-Kameradrehung über alle Punkte
# Phase B: Kamera stoppt, Layer für Layer ausblenden (außen -> innen)
# Phase C: nur Maskenpunkte einige Frames stehen lassen (keine Bewegung)

frames_A = 120   # langsamer als zuvor
frames_B = max(30, 2 * len(scat_layers))  # genug Frames für weichen Übergang
frames_C = 30
total_frames = frames_A + frames_B + frames_C

def update(i):
    if i < frames_A:
        # Phase A: Kameradrehung, alle Punkte sichtbar
        az = azim0 + 360.0 * (i / frames_A)
        ax_rgb.view_init(elev=elev0, azim=az)
        # sicherstellen, dass alle Layer sichtbar sind
        for s in scat_layers:
            s.set_visible(True)
        scat_in.set_visible(True)

    elif i < frames_A + frames_B:
        # Phase B: Kamera fixiert, Außen-Layer schrittweise ausblenden
        ax_rgb.view_init(elev=elev0, azim=azim0)
        progress = (i - frames_A) / frames_B  # 0 -> 1
        # Anzahl Layer, die bereits ausgeblendet sind
        num_off = int(np.floor(progress * len(scat_layers)))
        for k, s in enumerate(scat_layers):
            s.set_visible(False if k < num_off else True)
        scat_in.set_visible(True)

    else:
        # Phase C: nur Maskenpunkte, statische Kamera
        ax_rgb.view_init(elev=elev0, azim=azim0)
        for s in scat_layers:
            s.set_visible(False)
        scat_in.set_visible(True)

    # Rückgabe der Artists (blit=False, Rückgabe optional)
    return scat_layers + [scat_in]

anim = FuncAnimation(fig, update, frames=total_frames, interval=50, blit=False)

# ---------- Speichern ----------
output_path = "Visualisierung RGB GIF.gif"  # relativ zum Notebook-Verzeichnis
anim.save(output_path, writer=PillowWriter(fps=25))
print(f"GIF gespeichert: {output_path}")


<IPython.core.display.Javascript object>

GIF gespeichert: Visualisierung RGB GIF.gif
